商品名稱 'title'

商品圖片網址 是由images中的'base_url'及 'content_labels'中的'image_url'組成

商品價錢 'formatted_current_price'

商品文宣 'description'

商品評價 'average_rating'

商品評價分布 'rating_distribution' 會分別顯示'RatingValue'1-5的次數

商品3項指標得分數 'secondary_ratings_averages'包含 'Value', 'Quality', 'Taste'。分數是取裡面的'AverageRating'

商品瀏覽數 'total_reviews'

商品評論 'top_reviews'，裡面又包含'user_nickname','title','rating','review_text'

商品描述 'bullet_description' 包含 'Contains','Form','State of Readiness','Package Quantity','Package type','Net weight'

商品特色(店家描述) 'soft_bullets'中的'bullets'

為了取得商品中的label info ， 先透過json取得商品所在網址，再透過
商品_url = 'https://www.target.com'+'url網址' 進入商品頁面，再用request的方式取的label info 的資訊

### 爬一個頁面資料

In [70]:
import requests
from bs4 import BeautifulSoup
from requests import sessions
import pandas as pd
import json
import numpy as np

kw = 'cheese'
url = 'https://redsky.target.com/v2/plp/search/?channel=web&count=96&keyword={kw}&offset=0&pricing_store_id=3991&key=ff457966e64d5e877fdbad070f276d18ecec4a01'
data =  requests.get(url.format(kw=kw)).json()

#商品名稱
commodity_name = [i['title'] for i in data['search_response']['items']['Item']]

#商品圖片網址
commodity_images = [i['images'][0]['base_url']+i['images'][0]['primary'] for i in data['search_response']['items']['Item']]

#商品價錢

commodity_price = [i['price']['formatted_current_price'] for i in data['search_response']['items']['Item']]

#商品url
commodity_url = ['https://www.target.com'+i['url'] for i in data['search_response']['items']['Item']]

#商品文宣
commodity_description = []
for i in data['search_response']['items']['Item']:
    try:
        commodity_description.append(i['description'])
    except KeyError:
        commodity_description.append(np.nan)

#商品評價
commodity_average_rating = [i['average_rating'] for i in data['search_response']['items']['Item']]


#商品評價(次數)分布
#1星
commodity_RatingValue1 = [i['rating_distribution'][0]['Count'] for i in data['search_response']['items']['Item']]

#2星
commodity_RatingValue2 = [i['rating_distribution'][1]['Count'] for i in data['search_response']['items']['Item']]

#3星
commodity_RatingValue3 = [i['rating_distribution'][2]['Count'] for i in data['search_response']['items']['Item']]

#4星
commodity_RatingValue4 = [i['rating_distribution'][3]['Count'] for i in data['search_response']['items']['Item']]

#5星
commodity_RatingValue5 = [i['rating_distribution'][4]['Count'] for i in data['search_response']['items']['Item']]

#商品3項指標得分數  有些商品會沒有這些資料，應該移除
# Value
commodity_Value_score = []
for i in data['search_response']['items']['Item']:
    try:
        commodity_Value_score.append(i['secondary_ratings_averages']['Value']['AverageRating'])
    except KeyError:
        commodity_Value_score.append(np.nan)

#Quality
commodity_Quality_score = []
for i in data['search_response']['items']['Item']:
    try:
        commodity_Quality_score.append(i['secondary_ratings_averages']['Quality']['AverageRating'])
    except KeyError:
        commodity_Quality_score.append(np.nan)

#Taste
commodity_Taste_score = []
for i in data['search_response']['items']['Item']:
    try:
        commodity_Taste_score.append(i['secondary_ratings_averages']['Taste']['AverageRating'])
    except KeyError:
        commodity_Taste_score.append(np.nan)

#商品瀏覽數
commodity_total_reviews = [i['total_reviews'] for i in data['search_response']['items']['Item']]

#商品特色(店家描述)
commodity_highlights = []
for i in data['search_response']['items']['Item']:
    try:
        commodity_highlights.append(i['soft_bullets']['bullets'])
    except KeyError:
        commodity_highlights.append(np.nan)

#商品描述 'bullet_description'
commodity_descriptio = [str(i['bullet_description']).replace('<B>','').replace('</B>','').replace("'","") for i in data['search_response']['items']['Item']]

### 裝資料的容器

In [66]:
commodity_name = []
commodity_images = []
commodity_price = []
commodity_url = []
commodity_description = []
commodity_average_rating = []
commodity_RatingValue1 = []
commodity_RatingValue2 = []
commodity_RatingValue3 = []
commodity_RatingValue4 = []
commodity_RatingValue5 = []
commodity_Value_score = []
commodity_Quality_score = []
commodity_Taste_score = []
commodity_total_reviews = []
commodity_highlights = []
commodity_descriptio = []

In [61]:
kw = 'cheese'
number = 0
url = 'https://redsky.target.com/v2/plp/search/?channel=web&count=96&keyword={kw}&offset={number}&pricing_store_id=3991&key=ff457966e64d5e877fdbad070f276d18ecec4a01'
data =  requests.get(url.format(kw=kw,number=number)).json()
for i in data['search_response']['items']['Item']:
    p_price = lambda x:x['price'].get('formatted_current_price', np.nan)
    commodity_price.append(p_price(i))

### 爬多個頁面

In [1]:
import requests
from bs4 import BeautifulSoup
from requests import sessions
import pandas as pd
import json
import numpy as np
import random
import time

kw = 'cheese'
number = 0
while number < 3000:
    try:
        url = 'https://redsky.target.com/v2/plp/search/?channel=web&count=96&keyword={kw}&offset={number}&pricing_store_id=3991&key=ff457966e64d5e877fdbad070f276d18ecec4a01'
        
        data =  requests.get(url.format(kw=kw,number=number)).json()
        
        
        for i in data['search_response']['items']['Item']:
            commodity_name.append(i['title']) #商品名稱
            commodity_images.append(i['images'][0]['base_url']+i['images'][0]['primary'])#商品圖片網址
            commodity_url.append('https://www.target.com'+i['url']) #商品url
            commodity_average_rating.append(i['average_rating']) #商品評價
            commodity_RatingValue1.append(i['rating_distribution'][0]['Count']) #商品評價(次數)分布 #1星
            commodity_RatingValue2.append(i['rating_distribution'][1]['Count']) #商品評價(次數)分布 #2星
            commodity_RatingValue3.append(i['rating_distribution'][2]['Count']) #商品評價(次數)分布 #3星
            commodity_RatingValue4.append(i['rating_distribution'][3]['Count']) #商品評價(次數)分布 #4星
            commodity_RatingValue5.append(i['rating_distribution'][4]['Count']) #商品評價(次數)分布 #5星
            commodity_total_reviews.append(i['total_reviews']) #商品瀏覽數
            commodity_descriptio.append(str(i['bullet_description']).replace('<B>','').replace('</B>','').replace("'",""))  #商品描述 'bullet_description'
       
        #商品價錢
        for i in data['search_response']['items']['Item']:
            try:
                commodity_price.append(i['price']['formatted_current_price'])
            except:
                commodity_price.append(np.nan)
        
        #商品文宣
        for i in data['search_response']['items']['Item']:
            try:
                commodity_description.append(i['description'])
            except KeyError:
                commodity_description.append(np.nan)
        
        #商品3項指標得分數
        # Value
        
        for i in data['search_response']['items']['Item']:
            try:
                commodity_Value_score.append(i['secondary_ratings_averages']['Value']['AverageRating'])
            except KeyError:
                commodity_Value_score.append(np.nan)
        
        #Quality
        
        for i in data['search_response']['items']['Item']:
            try:
                commodity_Quality_score.append(i['secondary_ratings_averages']['Quality']['AverageRating'])
            except KeyError:
                commodity_Quality_score.append(np.nan)
        
        #Taste
        
        for i in data['search_response']['items']['Item']:
            try:
                commodity_Taste_score.append(i['secondary_ratings_averages']['Taste']['AverageRating'])
            except KeyError:
                commodity_Taste_score.append(np.nan)
        
        #商品特色(店家描述)
        
        for i in data['search_response']['items']['Item']:
            try:
                commodity_highlights.append(i['soft_bullets']['bullets'])
            except KeyError:
                commodity_highlights.append(np.nan)

        number += 96
        #time.sleep(random.uniform(1, 5))
    except KeyError: #如果爬取的url 沒有上方的資料，就會產生KeyError，因此break
        break

NameError: name 'commodity_name' is not defined

In [68]:
df = pd.DataFrame({"commodity_name":commodity_name,'commodity_images':commodity_images,'commodity_price':commodity_price,'commodity_url':commodity_url,'commodity_description':commodity_description
,'commodity_average_rating':commodity_average_rating,'commodity_RatingValue1':commodity_RatingValue1,'commodity_RatingValue2':commodity_RatingValue2,
'commodity_RatingValue3':commodity_RatingValue3,'commodity_RatingValue4':commodity_RatingValue4,'commodity_RatingValue5':commodity_RatingValue1,                   
'commodity_Value_score':commodity_Value_score,'commodity_Quality_score':commodity_Quality_score,'commodity_Taste_score':commodity_Taste_score,                 
'commodity_total_reviews':commodity_total_reviews,'commodity_highlights':commodity_highlights,'commodity_descriptio':commodity_descriptio              
                 
                 
                 
                 
                 
                 
                  })
df.isna().sum()

commodity_name                0
commodity_images              0
commodity_price               0
commodity_url                 0
commodity_description        45
commodity_average_rating      0
commodity_RatingValue1        0
commodity_RatingValue2        0
commodity_RatingValue3        0
commodity_RatingValue4        0
commodity_RatingValue5        0
commodity_Value_score       214
commodity_Quality_score     207
commodity_Taste_score       209
commodity_total_reviews       0
commodity_highlights         59
commodity_descriptio          0
dtype: int64

In [87]:
df.to_csv('./123.csv')

In [69]:
df.sort_values(by=['commodity_name'])  #會有相同的commodity_name	，但某些欄位不一樣

,commodity_name,commodity_images,commodity_price,commodity_url,commodity_description,commodity_average_rating,commodity_RatingValue1,commodity_RatingValue2,commodity_RatingValue3,commodity_RatingValue4,commodity_RatingValue5,commodity_Value_score,commodity_Quality_score,commodity_Taste_score,commodity_total_reviews,commodity_highlights,commodity_descriptio
491,1% Milkfat Small Curd Cottage Cheese - 16oz - ...,https://target.scene7.com/is/image/Target/GUES...,$1.99,https://www.target.com/p/1-milkfat-small-curd-...,Give your morning a flavorful start or power t...,3.00,3,0,0,0,3,3.50,2.33,3.00,6,"[Small-curd cottage cheese with 1% milk fat, M...","[Contains: Milk, May Contain: Soy, Tree Nuts, ..."
217,1% Milkfat Small Curd Cottage Cheese - 24oz - ...,https://target.scene7.com/is/image/Target/GUES...,$2.99,https://www.target.com/p/1-milkfat-small-curd-...,Whether you're looking for a quick breakfast o...,2.96,11,2,2,1,11,2.92,2.38,2.38,27,"[Contains 24oz of low-fat cottage cheese, Made...","[Contains: Milk, May Contain: Soy, Tree Nuts, ..."
285,4% Milkfat Small Curd Cottage Cheese - 24oz - ...,https://target.scene7.com/is/image/Target/GUES...,$2.99,https://www.target.com/p/4-milkfat-small-curd-...,With a container of 4% Milk Fat Small Curd Cot...,3.39,8,4,0,1,8,3.33,3.21,3.26,28,"[Small-curd cottage cheese with 4% milk fat, M...","[Contains: Milk, May Contain: Soy, Tree Nuts, ..."
746,8 Myles Frozen BBQ Mac and Cheese - 8oz,https://target.scene7.com/is/image/Target/GUES...,$4.49,https://www.target.com/p/8-myles-frozen-bbq-ma...,Whole grain elbow pasta tossed in a creamy whi...,5.00,0,0,0,0,0,5.00,5.00,5.00,3,"[Smokey BBQ Sauce, All real ingredients, Whole...","[Contains: Eggs, Milk, Wheat, Features: No Art..."
775,8 Myles Frozen BBQ Mac and Cheese - 8oz,https://target.scene7.com/is/image/Target/GUES...,$4.49,https://www.target.com/p/8-myles-frozen-bbq-ma...,Whole grain elbow pasta tossed in a creamy whi...,5.00,0,0,0,0,0,5.00,5.00,5.00,3,"[Smokey BBQ Sauce, All real ingredients, Whole...","[Contains: Eggs, Milk, Wheat, Features: No Art..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642,Whisps Multipack Parmesan - 6ct.,https://target.scene7.com/is/image/Target/GUES...,$6.59,https://www.target.com/p/whisps-multipack-parm...,Whisps Cheese Crisps are made by cheese people...,4.13,4,2,1,2,4,3.55,4.36,3.92,30,"[Whisps 6-count snack bags, 100% Parmesan Chee...","[Contains: Milk, Dietary Needs: Gluten Free, C..."
638,Whisps Parmesan &#38; Asiago and Pepper Jack C...,https://target.scene7.com/is/image/Target/GUES...,$7.49,https://www.target.com/p/whisps-parmesan-38-as...,NaN,5.00,0,0,0,0,0,NaN,NaN,NaN,1,NaN,"[Contains: Milk, Form: Pieces, State of Readin..."
89,Whisps Parmesan Cheese Crisps - 2.12oz,https://target.scene7.com/is/image/Target/GUES...,$3.99,https://www.target.com/p/whisps-parmesan-chees...,Whisps are made from just one delicious ingred...,4.17,9,2,8,7,9,3.38,4.35,4.00,78,[100% Parmesan Cheese - the only artisanal cop...,"[Contains: Milk, Dietary Needs: Gluten Free, F..."
188,Whisps Tangy Ranch Cheese Crisps - 2.12oz,https://target.scene7.com/is/image/Target/GUES...,$3.69,https://www.target.com/p/whisps-tangy-ranch-ch...,A cheddar-for-you Ranch! What happens when we ...,3.55,2,2,0,2,2,2.67,5.00,3.00,11,"[10g of Protein per Serving, 3g Carbs per Serv...","[Contains: Milk, Form: Pieces, State of Readin..."
